In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('my-linear-regression-code-along').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
df = spark.read.csv('Ecommerce-Customers.csv', inferSchema=True, header=True)

In [4]:
df.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [12]:
df.head(1)[0]

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

In [26]:
for item in df.head(4)[3]:
    print(item)

riverarebecca@gmail.com
1414 David ThroughwayPort Jason, OH 22070-1220
SaddleBrown
34.30555662975554
13.717513665142507
36.72128267790313
3.120178782748092
581.8523440352177


In [30]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [56]:
column = df.columns
column

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [59]:
columns = column[3:7]
columns

['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership']

In [60]:
assemlber = VectorAssembler(inputCols=columns, outputCol='features')

In [62]:
output = assemlber.transform(df)

In [64]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [65]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [68]:
final_data = output.select('features','Yearly Amount Spent')
final_data.head(5)

[Row(features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]), Yearly Amount Spent=587.9510539684005),
 Row(features=DenseVector([31.9263, 11.1095, 37.269, 2.664]), Yearly Amount Spent=392.2049334443264),
 Row(features=DenseVector([33.0009, 11.3303, 37.1106, 4.1045]), Yearly Amount Spent=487.54750486747207),
 Row(features=DenseVector([34.3056, 13.7175, 36.7213, 3.1202]), Yearly Amount Spent=581.8523440352177),
 Row(features=DenseVector([33.3307, 12.7952, 37.5367, 4.4463]), Yearly Amount Spent=599.4060920457634)]

In [69]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [70]:
print('train data: ')
train_data.describe().show()
print('test data: ')
test_data.describe().show()

train data: 
+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                336|
|   mean| 499.92822191264685|
| stddev|  79.30891286401912|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+

test data: 
+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                164|
|   mean|   498.055710772232|
| stddev|  79.55483132438899|
|    min|   266.086340948469|
|    max|  712.3963268096637|
+-------+-------------------+



In [72]:
print('train data: ')
train_data.head(5)


train data: 


[Row(features=DenseVector([29.5324, 10.9613, 37.4202, 4.0464]), Yearly Amount Spent=408.6403510726275),
 Row(features=DenseVector([30.4925, 11.5629, 35.9766, 1.4816]), Yearly Amount Spent=282.4712457199145),
 Row(features=DenseVector([30.5744, 11.351, 37.0888, 4.0783]), Yearly Amount Spent=442.06441375806565),
 Row(features=DenseVector([30.7377, 12.6366, 36.2138, 3.3578]), Yearly Amount Spent=461.7807421962299),
 Row(features=DenseVector([30.8364, 13.1001, 35.9077, 3.3616]), Yearly Amount Spent=467.5019004269896)]

In [73]:
print('test data: ')
test_data.head(5)

test data: 


[Row(features=DenseVector([30.3932, 11.803, 36.3158, 2.0838]), Yearly Amount Spent=319.9288698031936),
 Row(features=DenseVector([30.8162, 11.8514, 36.925, 1.0846]), Yearly Amount Spent=266.086340948469),
 Row(features=DenseVector([31.0662, 11.7351, 36.5994, 3.9589]), Yearly Amount Spent=448.93329320767435),
 Row(features=DenseVector([31.2681, 12.1325, 35.4568, 3.0721]), Yearly Amount Spent=423.4705331738239),
 Row(features=DenseVector([31.3585, 12.8099, 36.5497, 3.6377]), Yearly Amount Spent=495.1759504494754)]

In [74]:
lr = LinearRegression(featuresCol='features',labelCol='Yearly Amount Spent',predictionCol='prediction')

In [75]:
lr_model = lr.fit(train_data)

In [76]:
test_results = lr_model.evaluate(test_data)

In [77]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -12.93368875558565|
|-18.772330234257993|
|-13.068880355149929|
| -4.529125670358326|
|  3.429014065401475|
|  3.649491599129874|
|0.34960574328192706|
| -5.611870523543814|
| -17.92440721517505|
|-14.383604266891552|
|-1.9547072979972882|
| -6.386404591924759|
|-1.2935332888331459|
|  3.143110866635027|
|-18.970940150858326|
|  7.489453730563525|
|-10.095655581483186|
|-2.8783943028761314|
|-2.6458684949583358|
| -5.662290297945617|
+-------------------+
only showing top 20 rows



In [78]:
test_results.rootMeanSquaredError

8.84795691649801

In [79]:
test_results.r2

0.9875545924333453

In [80]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [103]:
prediction_data = lr_model.transform(final_data).show()

+--------------------+-------------------+------------------+
|            features|Yearly Amount Spent|        prediction|
+--------------------+-------------------+------------------+
|[34.4972677251122...|  587.9510539684005| 594.5438433024419|
|[31.9262720263601...|  392.2049334443264|  380.878971342408|
|[33.0009147556426...| 487.54750486747207| 504.8125925568138|
|[34.3055566297555...|  581.8523440352177| 570.9710950389097|
|[33.3306725236463...|  599.4060920457634| 591.4092507425132|
|[33.8710378793419...|   637.102447915074| 637.7740285052319|
|[32.0215955013870...|  521.5721747578274| 516.4122234001745|
|[32.7391429383803...|  549.9041461052942| 558.1962780789495|
|[33.9877728956856...|  570.2004089636196|  559.662445972574|
|[31.9365486184489...|  427.1993848953282|441.43968307123873|
|[33.9925727749537...|  492.6060127179966|509.53093573746946|
|[33.8793608248049...|  522.3374046069357| 513.2978135960332|
|[29.5324289670579...|  408.6403510726275|398.34434889233125|
|[33.190